# Tutorial on collocaing a daTASET  with lagged dat

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import warnings
# filter some warning messages
warnings.filterwarnings("ignore") 
from geopy.distance import geodesic 

####################you will need to change some paths here!#####################
#list of input files
#filename_bird='f:/data/project_data/NASA_biophysical/collocated_data/zoo_selgroups_HadSST_relabundance_5aug2019_plumchrusV_4regions_final.csv'
#output files
filename_bird='f:/data/project_data/NASA_biophysical/collocated_data/NPPSD_GOA_allseabird_wide_sat_and_eddy_data_final.nc'
filename_bird_out='f:/data/project_data/NASA_biophysical/collocated_data/NPPSD_GOA_allseabird_wide_sat_and_eddy_data_final_color.csv'
filename_bird_out_netcdf='f:/data/project_data/NASA_biophysical/collocated_data/NPPSD_GOA_allseabird_wide_sat_and_eddy_data_final_color.nc'
#################################################################################


## Reading CSV datasets

In [ ]:
#read in csv file in to panda dataframe & into xarray
#df_bird = pd.read_csv(filename_bird)
ds_bird = xr.open_dataset(filename_bird) #df_bird.to_xarray()
ilen_bird = len(ds_bird.lat)
ds_bird

In [ ]:
#clean up ds_bird so that zero are nan
cond= (ds_bird.cmc_sst>100)
clean = ds_bird.where(cond)
dlist = ['cmc_sst']
for var in dlist:
    ds_bird[var]=clean[var]
#plt.plot(ds_bird.time,ds_bird.cmc_sst,'b.')
plt.plot(clean.time,clean.cmc_sst,'r.')

In [ ]:
#clean up ds_bird so that zero are nan
cond= (ds_bird.ccmp_uwnd!=0)
clean = ds_bird.where(cond)
dlist = ['ccmp_uwnd','ccmp_vwnd']
for var in dlist:
    ds_bird[var]=clean[var]
plt.plot(ds_bird.time,ds_bird.ccmp_vwnd,'b.')
plt.plot(clean.time,clean.ccmp_vwnd,'r.')

In [ ]:
#clean up ds_bird so that zero are nan
cond= (ds_bird.aviso_adt>.01)
clean = ds_bird.where(cond)
dlist = ['aviso_adt','aviso_sla','aviso_ugos','aviso_ugosa','aviso_vgos','aviso_vgosa']
for var in dlist:
    ds_bird[var]=clean[var]
plt.plot(ds_bird.time,ds_bird.aviso_adt,'b.')
plt.plot(clean.time,clean.aviso_adt,'r.')

In [ ]:
#clean up ds_bird so that zero are nan
cond= (ds_bird.cyclonic_type>.1) | (ds_bird.cyclonic_type<-.1)
clean = ds_bird.where(cond)
dlist = ['observation_number','cyclonic_type','lat_eddy','lon_eddy','speed_average','speed_radius','track','save_eddy_distance']
for var in dlist:
    ds_bird[var]=clean[var]
plt.plot(ds_bird.time,ds_bird.observation_number,'b.')
plt.plot(clean.time,clean.observation_number,'r.')

In [ ]:
plt.plot(ds_bird.cyclonic_type,'.')

In [ ]:
ds_color = xr.open_dataset('https://rsg.pml.ac.uk/thredds/dodsC/CCI_ALL-v4.2-DAILY')
#ds_color = xr.open_dataset('https://rsg.pml.ac.uk/thredds/dodsC/CCI_ALL-v4.2-8DAY')
for var in ds_color:
    if not var=='chlor_a':
        ds_color = ds_color.drop(var)
ds_color

In [ ]:
#just check lat/lon & see looks okay
minlat,maxlat=ds_bird.lat.min(),ds_bird.lat.max()
minlon,maxlon=ds_bird.lon.min(),ds_bird.lon.max()
plt.scatter(ds_bird.lon,ds_bird.lat)
print(minlat.data,maxlat.data,minlon.data,maxlon.data)
print(ds_bird.time64.min().data,ds_bird.time64.max().data)

# Collocate all data with bird data

In [ ]:
ilen_bird = len(ds_bird.lat)
ds_data = ds_color
for var in ds_data:
    var_tem=var
    ds_bird[var_tem]=xr.DataArray(np.nan*np.empty(ilen_bird, dtype=str(ds_data[var].dtype)), coords={'index': ds_bird.index}, dims=('index'))
    ds_bird[var_tem].attrs=ds_data[var].attrs
print('var',var_tem)
for i in range(ilen_bird):
    t1,t2 = ds_bird.time64[i]-np.timedelta64(5,'D'), ds_bird.time64[i]+np.timedelta64(5,'D')
    lat1,lat2=ds_bird.lat[i]+.1,ds_bird.lat[i]-.1
    lon1,lon2=ds_bird.lon[i]-.1,ds_bird.lon[i]+.1
    tem = ds_data.sel(time=slice(t1,t2),lat=slice(lat1,lat2),lon=slice(lon1,lon2)).load()
    if len(tem)<1:
        continue
    tem = tem.mean(keep_attrs=True)
    for var in ds_data:
        var_tem=var
        ds_bird[var_tem][i]=tem[var].data
    if int(i/100)*100==i:
        print(i,len(ds_bird.lat))
#output data
df_bird = ds_bird.to_dataframe()
df_bird.to_csv(filename_bird_out)
ds_bird.to_netcdf(filename_bird_out_netcdf)

In [ ]:
plt.scatter(ds_bird.lon,ds_bird.lat,c=ds_bird.chlor_a)